In [11]:
from statsmodels.tsa.arima_model import ARIMAResults
import plotly
import plotly.graph_objs as go
from plotly.offline import *
import numpy as np
import pandas as pd
import csv

Load the model first

In [4]:
# LOAD LAST MODEL TO MAKE PREDICTIONS
# load model
results = ARIMAResults.load('AUSTemp_SARIMA.pkl')
print(results.summary())

                               SARIMAX Results                                
Dep. Variable:                      y   No. Observations:                 3649
Model:               SARIMAX(3, 1, 1)   Log Likelihood               -8376.942
Date:                Sat, 16 May 2020   AIC                          16763.884
Time:                        18:35:51   BIC                          16794.890
Sample:                             0   HQIC                         16774.927
                               - 3649                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1          0.4964      0.019     26.212      0.000       0.459       0.534
ar.L2         -0.1304      0.018     -7.096      0.000      -0.166      -0.094
ar.L3       5.042e-05      0.018      0.003      0.9

Make some predictions

In [7]:
# 4 days prediction with 95% confidence interval
nforecast = 4
predict = results.get_prediction(end=results.nobs+ nforecast-1, exog=None)
idx = np.arange(len(predict.predicted_mean))
predict_ci = predict.conf_int(alpha=0.5)

print(predict.predicted_mean)
print(predict_ci[:, 0].astype(int))
print(predict_ci[:, 1].astype(int))

[ 0.         25.83797676 15.3890801  ... 14.2370869  14.00358876
 13.97592616]
[-953 -689 -661 ...   12   12   12]
[953 740 692 ...  16  15  15]


Paint the predictions with confidence intervals

In [12]:
# read dataset and prepare data for the graph
with open("daily-min-temperatures.csv") as f:
    temp_df = pd.DataFrame(csv.DictReader(f))
temp_df['Date'] = pd.to_datetime(temp_df.Date)
temp_df.set_index('Date')
temp_df['Temp'] = pd.to_numeric(temp_df.Temp)

In [14]:
# finally generating the graph
actual = go.Scatter(
    x=temp_df.index,
    y=temp_df['Temp'].values,
    name = "actual",
    mode = 'markers',
    line = dict(color = '#0F2045'),
    opacity = 0.8)

predicted = go.Scatter(
    x=temp_df.index,
    y=predict.predicted_mean,
    name = "predicted",
    line = dict(color = '#0F91CF', dash = 'dash'),
    opacity = 0.8)

predicted_low = go.Scatter(
    x=temp_df.index,
    y=predict_ci[:, 0],
    mode='none',
    name = "low_ci",
    fill = 'tonexty',
    fillcolor='rgba(145, 217, 232, 0.7)')

predicted2 = go.Scatter(
    x=temp_df.index,
    y=predict.predicted_mean,
    name = "predicted2",
    line = dict(color = '#0F91CF', dash = 'dash'),
    opacity = 0.8,
    showlegend = False)

predicted_high = go.Scatter(
    x=temp_df.index,
    y=predict_ci[:, 1],
    mode='none',
    name = "high_ci",
    fill = 'tonexty',
    fillcolor='rgba(145, 217, 232, 0.7)')

data_fig = [actual, predicted, predicted_low, predicted2, predicted_high]

layout = go.Layout(title="Model 16th May 2020 - Temperatures",
                xaxis=dict(title='Day'),
                yaxis=dict(title='Temperature'))

fig = go.Figure(data=data_fig, layout=layout)
plotly.offline.plot(fig, filename='AUSTemp_SARIMA.html')

'AUSTemp_SARIMA.html'

Now you can go to your local folder and open the file in a browser. You can select and play with the graph.